# Code notes for BCCNet

source: https://github.com/OlgaIsupova/BCCNet

**utils**: https://github.com/OlgaIsupova/BCCNet/blob/master/utils/utils_dataset_processing.py <br><br>
Two functions: 
1. shuffle_arrays(arrays)
2. shrink_arrays(arrays, shrink_size, is_shuffle=True)

In [3]:
import numpy as np

In [49]:
# assume that our data is in 3d, i.e.for each sample we have 2d input.
# for instance, (6000,28,28) for MNIST
# what the shuffle does is to pick out the rows, and random shuffle those rows

def shuffle_arrays(arrays):
    #if not arrays:
    #    return arrays

    size = arrays[0].shape[0]
    permutation = np.arange(size)
    np.random.shuffle(permutation)
    for i, array in enumerate(arrays):
        arrays[i] = array[permutation]

    return arrays

In [59]:
np.random.seed(100)
x = np.random.choice([0,1], size=(6000, 4, 4))
print(x[0])
print(shuffle_arrays(x)[0])

[[0 0 1 1]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 0 0]]
[[0 0 0 1]
 [0 0 0 0]
 [0 0 1 1]
 [1 1 0 0]]


In [60]:
# what we do is shuffle rows like the previous function
# then output a tuple which contains partial and whole of the example


def shrink_arrays(arrays, shrink_size, is_shuffle=True):
    # if shrink_size in [0.0, 1.0] it specifies fraction of the array size to be extracted, if shrink_size is an
    # integer it specifies the size of the shrunk arrays

    #if not arrays:
    #    return arrays

    if type(shrink_size) == float or type(shrink_size) == np.float64:
        assert(0.0 <= shrink_size <= 1.0)
        size = arrays[0].shape[0]
        shrunk_array_size = int(round(shrink_size * size))
    else:
        shrunk_array_size = shrink_size

    if is_shuffle:
        shuffled_arrays = shuffle_arrays(arrays)
    else:
        shuffled_arrays = arrays

    shrunk_arrays = []
    for array in shuffled_arrays:
        shrunk_arrays.append(array[:shrunk_array_size])

    return shrunk_arrays, shuffled_arrays

In [64]:
np.random.seed(100)
print(x[0])
labelled_train, whole_train = shrink_arrays(x, 0.25) #shrunk_arrays, shuffled_arrays
print(labelled_train[0])
print(whole_train[0])

[[0 0 0 0]
 [1 1 0 0]
 [0 0 1 1]
 [0 0 0 1]]
[[0 0 1 1]]
[[0 0 1 1]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 0 0]]


np function: **numpy.expand_dims()**

In [79]:
np.random.seed(100)
print(x[0])
y = np.expand_dims(x, axis=3)
print(y[0])
print(y.shape)

[[0 0 1 1]
 [1 1 0 0]
 [0 0 0 1]
 [0 0 0 0]]
[[[0]
  [0]
  [1]
  [1]]

 [[1]
  [1]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  [1]]

 [[0]
  [0]
  [0]
  [0]]]
(6000, 4, 4, 1)


In [24]:
n_classes = 3
reliability_level = 0.6
alpha = (reliability_level - (1 - reliability_level) / (n_classes - 1)) * np.eye(n_classes) + \
            (1 - reliability_level) / (n_classes - 1)
print(alpha)

confusion_matrices = np.zeros((3, 3, 5), dtype=np.float64)
for k in np.arange(5):
    for i in np.arange(3):
        confusion_matrices[i, :, k] = np.random.dirichlet(alpha[i, :])
print(confusion_matrices)


x = np.cumsum(confusion_matrices[:, :, 1], axis=1)
print(x)









[[0.6 0.2 0.2]
 [0.2 0.6 0.2]
 [0.2 0.2 0.6]]
[[[2.60443695e-01 1.50222467e-01 1.64188820e-01 6.35220479e-01
   9.06720061e-01]
  [7.08570616e-01 1.24915813e-09 4.19045135e-07 2.60291192e-06
   9.28793808e-02]
  [3.09856886e-02 8.49777532e-01 8.35810761e-01 3.64776918e-01
   4.00558549e-04]]

 [[8.57248735e-02 8.09221819e-05 6.90062379e-01 6.18002044e-01
   3.69328193e-02]
  [9.14275117e-01 9.99471865e-01 3.09904988e-01 3.46448610e-03
   9.43204722e-01]
  [9.41801356e-09 4.47213280e-04 3.26332918e-05 3.78533470e-01
   1.98624584e-02]]

 [[1.86274606e-02 1.76781772e-04 4.43271386e-01 2.95320063e-01
   2.63320486e-04]
  [9.76598597e-03 3.12931161e-02 5.01200339e-02 5.51499504e-05
   6.55430081e-02]
  [9.71606553e-01 9.68530102e-01 5.06608580e-01 7.04624787e-01
   9.34193671e-01]]]
[[1.50222467e-01 1.50222468e-01 1.00000000e+00]
 [8.09221819e-05 9.99552787e-01 1.00000000e+00]
 [1.76781772e-04 3.14698979e-02 1.00000000e+00]]


In [7]:
x = np.random.binomial(1, 0.8, (10, 5))
print(x)

y = np.random.rand(10,1)
print(y)




[[1 1 1 0 1]
 [1 1 1 1 0]
 [1 1 0 1 1]
 [0 1 1 1 1]
 [1 1 1 0 0]
 [0 0 1 1 0]
 [1 1 0 0 1]
 [1 1 1 1 1]
 [1 0 0 1 1]
 [1 1 1 0 1]]
[[0.21465633]
 [0.41011371]
 [0.55321564]
 [0.95173549]
 [0.87471379]
 [0.85582064]
 [0.75820016]
 [0.89383032]
 [0.68797276]
 [0.87556469]]
